In [4]:
from collections import defaultdict
import random


def reservoir_sampling_with_max_occurrence(stream, k):
    """
    Perform reservoir sampling on a stream of (item, occurrence) pairs,
    keeping only the items with the maximum occurrence.
    
    :param stream: An iterator of (item, occurrence) pairs
    :param k: The maximum number of items to sample
    :return: A sample containing items with the maximum occurrence
    """
    if k <= 0:
        return []

    # Initialize the reservoir and the maximum occurrence seen so far
    reservoir = []
    max_occurrence = 0
    occurrence_dict = defaultdict(int)
    
    for i, (item, occurrence) in enumerate(stream):
        occurrence_dict[item] += occurrence
        current_occurrence = occurrence_dict[item]
        
        # If the current item's occurrence is greater than the max observed,
        # clear the reservoir and start a new one
        if current_occurrence > max_occurrence:
            max_occurrence = current_occurrence
            reservoir = [(item, current_occurrence)]
        
        # If the current item's occurrence is equal to the max, add it to the reservoir
        elif current_occurrence == max_occurrence:
            reservoir.append((item, current_occurrence))
        
        # Ensure the reservoir size is not exceeded
        while len(reservoir) > k:
            # Random eviction from the reservoir
            eviction_index = random.randrange(len(reservoir))
            evicted_item, _ = reservoir.pop(eviction_index)
            # Ensure that if we popped the last occurrence of an item, we reduce the max_occurrence value
            if all(item != evicted_item for item, _ in reservoir):
                max_occurrence = max(occurrence for _, occurrence in reservoir) if reservoir else 0

    return reservoir

# Example usage:
stream_data = [('a', 1), ('b', 1), ('d', 1), ('e', 1), ('f', 1), ('g', 1), ('h', 1), ('i', 4), ('j', 3)]
sample_size = 3
sample = reservoir_sampling_with_max_occurrence(stream_data, sample_size)
print("The sample with maximum occurrence is:", sample)


The sample with maximum occurrence is: [('i', 4)]


In [11]:
import random
import math

def weighted_reservoir_sampling(stream, k):
    """Select k items from a weighted stream where stream is a sequence of (item, occurrence) pairs."""
    reservoir = []
    
    for item, occurrence in stream:
        if occurrence <= 0:
            continue  # Skip items with non-positive weight
        
        if len(reservoir) < k:
            # The reservoir is not yet full, add the item directly
            reservoir.append((item, occurrence))
        else:
            # Calculate the key for the current item
            key = random.random() ** (1 / occurrence)
            
            # Find the smallest key in the reservoir
            min_key_idx = min(range(k), key=lambda i: reservoir[i][1])
            min_key = reservoir[min_key_idx][1]
            
            # Replace the item with the smallest key if its key is smaller than the current item's key
            if min_key < key:
                reservoir[min_key_idx] = (item, occurrence)
    
    # Return just the items, not their occurrences
    return [item for item, occurrence in reservoir]

# Example usage:
stream_data = [ ('item2', 50), ('item3', 20), ('item4', 40),('item1', 10)]
selected_items = weighted_reservoir_sampling(stream_data, 2)
print(selected_items)

['item2', 'item3']


In [14]:
def reservoir_sampling_max(stream_items, k):
    """
    Reservoir sampling function to select items with maximum occurrence in a stream.

    :param stream_items: A list of tuples, where each tuple consists of an item and its occurrence.
    :param k: The number of items to sample.
    :return: A list containing the most frequently occurring items.
    """
    if k <= 0:
        return []

    # Initialize reservoir and occurrence count
    reservoir = []
    occurrences = defaultdict(int)

    # Process stream items
    for index, (item, count) in enumerate(stream_items):
        if len(reservoir) < k:
            # Fill the reservoir with initial items
            occurrences[item] += count
            reservoir.append((item, occurrences[item]))
        else:
            occurrences[item] += count
            # Check if the current item has more occurrences than the least in the reservoir
            current_min = min(reservoir, key=lambda x: x[1])
            if occurrences[item] > current_min[1]:
                # Replace the least frequent item in reservoir with the current item
                reservoir.remove(current_min)
                reservoir.append((item, occurrences[item]))

    # Sort the reservoir based on occurrence to return the most frequent items
    reservoir.sort(key=lambda x: x[1], reverse=True)
    return reservoir
    # Just return the items without their occurrence count
    return [item for item, _ in reservoir]


In [118]:
    stream = [('apple', 5), ('banana', 5), ('orange', 5), ('grape', 5), ('peach', 4),('mango', 5)]
    k =2
    print(reservoir_sampling_max(stream, k))

[('apple', 5), ('banana', 5)]


In [19]:
def weighted_reservoir_sampling(stream, k):
    # Initialize an empty reservoir with k slots
    reservoir = []
    total_occurrences = 0
    
    for item, occurrence in stream:
        total_occurrences += occurrence
        
        for _ in range(occurrence):
            if len(reservoir) < k:
                # Fill the reservoir with the first k items
                reservoir.append(item)
            else:
                # Randomly replace items in the reservoir with a new item
                # Probability is weighted based on occurrence of new items
                r = random.randint(1, total_occurrences)
                if r <= occurrence:
                    replace_index = random.randrange(k)
                    reservoir[replace_index] = item
    
    return reservoir
# Example usage:
# Create a list of (item, occurrence) pairs
item_stream = [('apple', 10), ('banana', 5), ('cherry', 20), ('date', 1), ('elderberry', 2)]
# Define the number of items to select
k_items = 3
# Execute the weighted reservoir sampling algorithm
selected_items = weighted_reservoir_sampling(item_stream, k_items)
# Show the results
print("Selected items:", selected_items)

Selected items: ['cherry', 'cherry', 'cherry']


In [68]:
import random
from typing import List, Tuple
def weighted_random_sample(stream: List[Tuple[int, int]], k: int) -> List[int]:
    """
    Selects k random items from a stream, weighted towards maximum occurrences.
    
    :param stream: A list of (item, occurrence) tuples.
    :param k: The number of items to select.
    :return: A list of selected items.
    """
    # Initialize the reservoir with the first k items from the stream
    reservoir = []
    total_weight = 0
    for item, weight in stream:
        total_weight += weight
        # If the reservoir isn't full, add the item directly
        if len(reservoir) < k:
            reservoir.append((item, weight))
        else:
            # Replace elements in the reservoir with a probability
            # proportional to the weight of the new item
            prob = weight / total_weight
            if random.random() < prob:
                # Choose an index proportional to weights for replacement
                weights = [w for _, w in reservoir]
                index_to_replace = weighted_choice(weights)
                reservoir[index_to_replace] = (item, weight)
    # Extract final items from the reservoir, disregarding weights at this point
    return [item for item, _ in reservoir]
def weighted_choice(weights: List[int]) -> int:
    """
    Make a weighted choice of an index.
    
    :param weights: A list of weights, one per item.
    :return: The selected index.
    """
    cumulative_weights = []
    total_sum = 0
    for w in weights:
        total_sum += w
        cumulative_weights.append(total_sum)
    rnd = random.random() * total_sum
    for i, total in enumerate(cumulative_weights):
        if rnd < total:
            return i
    return len(weights) - 1
# Example usage:
stream_data = [ ('elderberry', 25),
    ('cherry', 20),
    ('date', 30) , ('fig', 1) , ('apple', 100), ('banana', 5), ('mane', 100),
]
k_items = 3
print(weighted_random_sample(stream_data, k_items))

['elderberry', 'cherry', 'apple']


In [70]:
import pandas as pd

In [110]:
df = pd.read_csv('../data/sample.csv')
df.sample(100)

,desc_md5_cs,doc_occur_count
35,3bf3f29e4ff2cb63259bc1b9708ac773,1
27,b33beaf14e53fc3ee527b3d2b63ab58f,1
67,6d4df03ed301f52ef2ecde0a79a3d614,5
60,ecd6def544b034582e4ea559b592b7f5,1
99,954e356135cbb03339c2d967af368f60,7
...,...,...
26,5b9e26ac75709f669e7e4ca754882c17,2
53,314c1e5831fffc11c2e3024b8fe67702,1
3,27e42fdcb6ec8710cb0a6f9c1082629d,8
23,113fcd810a0c540a105eea0adb153e47,1


In [73]:
sample_list = []
for idx , data in df.iterrows():
    sample_list.append((data['desc_md5_cs'],data['doc_occur_count']))

In [74]:
sample_list[:10]

[('490382a5831a606f01e89bfe0d074d34', 1),
 ('dac826cb225e7b24fb7baf1c56e0f02e', 2),
 ('7da5fd42e13c18b74f1b5d4f4e894cc0', 4),
 ('27e42fdcb6ec8710cb0a6f9c1082629d', 8),
 ('660d2a4168c4ee2596339165cb5b4e87', 8),
 ('d41bcc2b6ddb3e5c0577ecf69de681b3', 14),
 ('f4090caf6e4ea3d69e561d7f1feecfb2', 1),
 ('b61c5f9b6cdd97d8a68e62d0e7e05652', 3),
 ('d18349304346479aeed8f7dc73857249', 3),
 ('da15cf8dbb32f8a1f8972580eca0c48a', 4)]

In [80]:
import numpy as np
from scipy import stats
import random
# Function to perform reservoir sampling
def reservoir_sampling(stream, k):
    reservoir = []
    for i, item in enumerate(stream):
        if i < k:
            reservoir.append(item)
        else:
            j = random.randrange(i + 1)
            if j < k:
                reservoir[j] = item
    return reservoir
# Function to split K and perform sampling
def split_and_sample(stream, k):
    items = []
    frequencies = []
    
    # Read the stream and collect items and frequencies
    for item, frequency in stream:
        items.append(item)
        frequencies.append(frequency)
    
    # Calculate the median and MAD
    median = np.median(frequencies)
    mad = stats.median_abs_deviation(frequencies)
    # mad = np.median(np.abs(frequencies - median))
    
    # Split items based on whether they are within the (median-MAD, median+MAD) range
    in_range_items = [(item, freq) for item, freq in zip(items, frequencies) if median - mad <= freq <= median + mad]
    out_range_items = [(item, freq) for item, freq in zip(items, frequencies) if freq < median - mad or freq > median + mad]
    
    # Split K into K1 and K2
    total_items = len(items)
    k1 = len(in_range_items) / total_items * k
    k2 = k - k1
    
    # Round K1 and K2 since they need to be integers, this can slightly alter their sum
    k1, k2 = round(k1), round(k2)
    # Perform reservoir sampling for in-range and out-range item frequencies
    l1 = reservoir_sampling(in_range_items, k1)
    l2 = reservoir_sampling(out_range_items, k2)
    # Merge samples from both ranges
    final_sample = l1 + l2
    
    return final_sample
# Example usage with a dummy stream and K value
# Assuming stream is given in the format [(item1, freq1), (item2, freq2), ...]
stream_data = [('apple', 10), ('banana', 20), ('cherry', 15), ('date', 25), ('elderberry', 30), ('fig', 40), ('grape', 5)]

k_value = 5
sample = split_and_sample(sample_list, k_value)
print("Final selected sample list:", sample)

Final selected sample list: [('4970805e81104b3a2ae01ebb35dc122b', 3), ('045e3113f113eac5b4b35395f177fee5', 1), ('d6abb492c25a8fce560a3f3cf6337ab5', 1), ('59d447e616d08bc50df2b0fc83e8ec03', 1), ('e813a019c13a1e3c0874ca6558cc160e', 8), ('da40d73d095ac29959f4f4e1ce78a98c', 8)]


In [105]:
import numpy as np
from scipy import stats
import random

class SamplingOnFrequency:

    def __init__(self):
        pass

    def reservoir_sampling(self, stream, k):
        reservoir = []
        for i, item in enumerate(stream):
            if i < k:
                reservoir.append(item)
            else:
                j = random.randrange(i + 1)
                if j < k:
                    reservoir[j] = item
        return reservoir

    def split_and_sample(self, stream, k):
        items = []
        frequencies = []
        
        # Read the stream and collect items and frequencies
        for item, frequency in stream:
            items.append(item)
            frequencies.append(frequency)
        
        # Calculate the median and MAD
        median = np.median(frequencies)
        mad = stats.median_abs_deviation(frequencies)
        print(f' the median is : {median} \n The median_abs_deviation  : {mad}\n')
        # mad = np.median(np.abs(frequencies - median))
        
        # Split items based on whether they are within the (median-MAD, median+MAD) range
        in_range_items = [(item, freq) for item, freq in zip(items, frequencies) if median - mad <= freq <= median + mad]
        out_range_items = [(item, freq) for item, freq in zip(items, frequencies) if freq < median - mad or freq > median + mad]
        print(f' total in range of mad : {len(in_range_items)} \n total out rang of mad  : {len(out_range_items)}\n')
        
        # Split K into K1 and K2
        total_items = len(items)
        k1 = len(in_range_items) / total_items * k
        k2 = k - k1
        
        # Round K1 and K2 since they need to be integers, this can slightly alter their sum
        k1, k2 = round(k1), round(k2)
        
        if k1+k2 > k:
            if k1>k2:
                k1 = k1 - ((k1+k2)-k)
            else:
                k2 = k2 - ((k1+k2)-k)
        if k1 +k2 < k:
            
            if k1<k2:
                k1 = k1 + (k-(k1+k2))
            else:
                k2 = k2 + (k-(k1+k2))
            
        print(f' value of k1 : {k1} \n value of k2 : {k2}\n')
        # Perform reservoir sampling for in-range and out-range item frequencies
        if len(in_range_items) >= len(out_range_items):
            k_in_range = min(k1,k2)
            k_out_range = max(k1,k2)
        else:
            k_in_range = max(k1,k2)
            k_out_range = min(k1,k2)
        print(f' value of k_in_range : {k_in_range} \n value of k_out_range : {k_out_range}\n')
        l1 = self.reservoir_sampling(in_range_items, k_in_range)
        l2 = self.reservoir_sampling(out_range_items, k_out_range)
        # Merge samples from both ranges
        final_sample = l1 + l2
        
        return final_sample


In [107]:
k_value = 20

sampling_occurrence = SamplingOccurrence()
sampled_data = sampling_occurrence.split_and_sample(sample_list, k=k_value)

# sample = split_and_sample(sample_list, k_value)
print("Final selected sample list:", sampled_data)

 the median is : 3.0 
 The median_abs_deviation  : 2.0

 total in range of mad : 70 
 total out rang of mad  : 30

 value of k1 : 14 
 value of k2 : 6

 value of k_in_range : 6 
 value of k_out_range : 14

Final selected sample list: [('5b9e26ac75709f669e7e4ca754882c17', 2), ('ca06cdc7fa9f7cfe166f5e0503a12fa1', 1), ('3c4543b5ba3bd204ba2e358e46f247e8', 4), ('4ece7e3dc26049f61f2bf76ddbe7c02c', 1), ('13905e07ce1aa2b16e7195cfec044466', 1), ('f88f9b5c39e1978976da2679586cc6dd', 2), ('27e42fdcb6ec8710cb0a6f9c1082629d', 8), ('599436458fdd30cb2c37557f155ee351', 40), ('ecdb1afc9093011c754ff7fd75e89931', 12), ('c846c2a697d52433208e3aab667f66f2', 18), ('82bf86d6803620758ae36277355d65c6', 18), ('da40d73d095ac29959f4f4e1ce78a98c', 8), ('17477d0ba44e8e4f575a52a64c536b3b', 6), ('24902be80e22259bfddec35bd1ee2c1a', 21), ('ac756914a07fc8def9ac8b382a7b53e0', 6), ('a678a582f10d679a2e4ee12a4b99a378', 8), ('cf89b24f46dbf13b46ee07d3640979e6', 27), ('c5102c3d54c25eca0923d5ace9fae54c', 12), ('390c72243b1de67a11

In [86]:
df.describe()

,doc_occur_count
count,100.000000
mean,5.270000
std,6.584118
min,1.000000
25%,1.000000
50%,3.000000
75%,6.250000
max,40.000000


In [94]:
df[df['doc_occur_count'].between(2,5)]

,desc_md5_cs,doc_occur_count
1,dac826cb225e7b24fb7baf1c56e0f02e,2
2,7da5fd42e13c18b74f1b5d4f4e894cc0,4
7,b61c5f9b6cdd97d8a68e62d0e7e05652,3
8,d18349304346479aeed8f7dc73857249,3
9,da15cf8dbb32f8a1f8972580eca0c48a,4
11,e6dca5b300af7b08c5b0be4140e1757d,4
12,4a4d20ca6aa10b955b8f74b6e1108de1,2
13,3c4543b5ba3bd204ba2e358e46f247e8,4
14,5130177e8483780f1e82d9ad80a55bb6,3
18,f88f9b5c39e1978976da2679586cc6dd,2


In [91]:
stats.median_abs_deviation(df['doc_occur_count'])

2.0

In [134]:
import random
import statistics
class StatsSampling:
    def __init__(self, data_stream, k):
        # initializing variables
        self.data_stream = data_stream
        self.k = k
    def get_median_mad(self, freq_list):
        median = statistics.median(freq_list)
        mad = statistics.median([abs(x - median) for x in freq_list])
        return median, mad
    def run_sampler(self):
        # Step 1: Calculate the median and MAD
        freq_list = [freq for _, freq in self.data_stream]
        median, mad = self.get_median_mad(freq_list)
        within_range = []
        outside_range = []
        # Step 2: Classify items based on (median-MAD) and (median+MAD)
        for item, freq in self.data_stream:
            if median - mad <= freq <= median + mad:
                within_range.append((item, freq))
            else:
                outside_range.append((item, freq))
        # Calculate K1 and K2
        total_items = len(self.data_stream)
        k1 = round(len(within_range) / total_items * self.k)
        k2 = self.k - k1
        # Step 3: Apply reservoir sampling for both groups
        if k1+k2 > self.k:
            if k1>k2:
                k1 = k1 - ((k1+k2)-self.k)
            else:
                k2 = k2 - ((k1+k2)-self.k)
        if k1 +k2 < k:
            
            if k1<k2:
                k1 = k1 + (self.k-(k1+k2))
            else:
                k2 = k2 + (self.k-(k1+k2))
            
        print(f' value of k1 : {k1} \n value of k2 : {k2}\n')
        # Perform reservoir sampling for in-range and out-range item frequencies
        if len(within_range) >= len(outside_range):
            k_in_range = min(k1,k2)
            k_out_range = max(k1,k2)
        else:
            k_in_range = max(k1,k2)
            k_out_range = min(k1,k2)
        print(f' value of k_in_range : {k_in_range} \n value of k_out_range : {k_out_range}\n')
        l1 = self.reservoir_sampling(within_range, k_in_range)
        l2 = self.reservoir_sampling(outside_range, k_out_range)
        # Step 4: Merge selected samples
        final_sample = l1 + l2
        return final_sample
    
    def reservoir_sampling(self, data, k):
        if len(data) <= k:
            return [(item,_) for item, _ in data]
        reservoir = [(item,_ )for item, _ in data[:k]]
        # Start from the (k+1)th element
        for i, (item, _) in enumerate(data[k:], start=k):
            j = random.randrange(0, i + 1)
            if j < k:
                reservoir[j] = (item,_)
        return reservoir
# Example usage:
data_stream = [
    ('item1', 10), 
    ('item2', 20), 
    ('item3', 10), 
    ('item4', 30), 
    ('item5', 40),
]
k_value = 20
reservoir_sampler = StatsSampling(sample_list, k=k_value)
final_sample = reservoir_sampler.run_sampler()
print(final_sample)

 value of k1 : 14 
 value of k2 : 6

 value of k_in_range : 6 
 value of k_out_range : 14

[('1735530eb501a1e3acc3a06371edef7a', 1), ('dac826cb225e7b24fb7baf1c56e0f02e', 2), ('4ece7e3dc26049f61f2bf76ddbe7c02c', 1), ('59d447e616d08bc50df2b0fc83e8ec03', 1), ('b8e8fb292ec44b740a121ae8c1365324', 3), ('7992bb78399476c015ec365b421ce230', 1), ('27e42fdcb6ec8710cb0a6f9c1082629d', 8), ('660d2a4168c4ee2596339165cb5b4e87', 8), ('e813a019c13a1e3c0874ca6558cc160e', 8), ('7b5b9ecad352f613c94473758a3e71ff', 24), ('a678a582f10d679a2e4ee12a4b99a378', 8), ('59a3f3ff480fef5ca070629316b7ff8a', 7), ('0e583437801fe6c70d7cff3b52781728', 8), ('99c6e75da6b4cbb64d1178352069141e', 6), ('ac756914a07fc8def9ac8b382a7b53e0', 6), ('514a0ed09c10a3575364573b75d94c7e', 26), ('ecdb1afc9093011c754ff7fd75e89931', 12), ('c5102c3d54c25eca0923d5ace9fae54c', 12), ('390c72243b1de67a11ced56b913bcfe8', 6), ('4f563abf01f5a21bddd409fe4a59789e', 8)]


In [132]:
reservoir_sampler.get_median_mad(df['doc_occur_count'])

(3.0, 2.0)

In [1]:
import math

In [4]:
math.log(2324,10)

3.3662361237182927